# 数据预处理

创建一个人工数据集，并存储在CSV（逗号分隔值）文件

In [1]:
import os
# 参数exist_ok=True表示如果目标目录已经存在，函数将不会抛出异常，而是无操作地结束。
os.makedirs(os.path.join('..', 'data'), exist_ok=True)
data_file = os.path.join('..', 'data', 'house_tiny.csv')
with open(data_file, 'w') as f:
    f.write('NumRooms,Alley,Price\n')
    f.write('NA,Pave,127500\n')
    f.write('2,NA,106000\n')
    f.write('4,NA,178100\n')
    f.write('NA,NA,140000\n')

从创建的CSV文件中加载原始数据集

In [3]:
import pandas as pd

data = pd.read_csv(data_file)
print(data)

   NumRooms Alley   Price
0       NaN  Pave  127500
1       2.0   NaN  106000
2       4.0   NaN  178100
3       NaN   NaN  140000


“NaN”项代表缺失值。 为了处理缺失的数据，典型的方法包括插值法和删除法， 其中插值法用一个替代值弥补缺失值，而删除法则直接忽略缺失值。
这里，我们将考虑插值法

pandas中.iloc是基于整数位置的选择，而.loc是基于标签的选择。使用如下:
```python
# 选择第一行到第三行（不包括第四行）
df.iloc[0:3]
# 选择第一行和第三行，第一列和第二列
df.iloc[[0, 2], [0, 1]]
# 选择标签为'row1'到'row3'的行（包括'row3'）
df.loc['row1':'row3']
# 选择标签为'row1'和'row3'的行，'col1'和'col2'的列
df.loc[['row1', 'row3'], ['col1', 'col2']]
```

In [7]:
inputs, outputs = data.iloc[:, 0:2], data.iloc[:, 2]
inputs['NumRooms'] = inputs['NumRooms'].fillna(inputs['NumRooms'].mean())
print(inputs)

   NumRooms Alley
0       3.0  Pave
1       2.0   NaN
2       4.0   NaN
3       3.0   NaN


对于`inputs`中的类别值或离散值，我们将“NaN”视为一个类别

.get_dummies将分类变量转换为称为"哑变量"或"虚拟变量"的一系列二进制列，每个唯一的类别值都会被分配一个新的列。如果原始数据中的某个特征有三个类别，get_dummies将这个特征转换成三列，每列对应一个类别，类别存在时为1，不存在时为0。

dummy_na=True：这个参数指定了是否为NaN值添加一个哑变量列。如果设置为True，那么每个特征中的NaN值也会被视作一个有效的类别，并为其创建一个新的列，其中NaN存在时为1，否则为0。

当使用哑变量时，要注意"虚拟变量陷阱"（Dummy Variable Trap），即高度相关（多重共线性）的特征列可能会导致一些机器学习模型的性能下降。为避免这个问题，一种做法是使用drop_first=True参数，这将从每个特征的哑变量中移除第一列，减少特征之间的完全多重共线性。

In [8]:
inputs = pd.get_dummies(inputs, dummy_na=True)
print(inputs)

   NumRooms  Alley_Pave  Alley_nan
0       3.0        True      False
1       2.0       False       True
2       4.0       False       True
3       3.0       False       True


现在`inputs`和`outputs`中的所有条目都是数值类型，它们可以转换为张量格式

In [9]:
import torch
# to_numpy() 方法将 DataFrame 转换为 NumPy 数组，然后再使用 torch.tensor() 函数将 NumPy 数组转换为张量。
X = torch.tensor(inputs.to_numpy(dtype=float))
y = torch.tensor(outputs.to_numpy(dtype=float))
X, y

(tensor([[3., 1., 0.],
         [2., 0., 1.],
         [4., 0., 1.],
         [3., 0., 1.]], dtype=torch.float64),
 tensor([127500., 106000., 178100., 140000.], dtype=torch.float64))

### 练习
1.删除缺失值最多的列。

2.将预处理后的数据集转换为张量格式。

df.dropna(axis=1, inplace=True) 是用于删除包含缺失值的整列数据。它会删除所有包含缺失值的列。axis=1 表示按列操作,inplace=True 参数是用于指示在原始数据上进行操作。

df.drop(column_with_most_missing_values, axis=1, inplace=True) 是根据变量 column_with_most_missing_values 的值来指定要删除的列名。它只会删除指定的那一列。同样，axis=1 表示按列操作。

In [19]:
import pandas as pd
import numpy as np
import torch

# 创建一个包含更多行和列的示例数据集
data = {
    '特征1': [1, 2, np.nan, 4, 5, 6],
    '特征2': [7, 8, 9, np.nan, 11, 12],
    '特征3': [13, 14, 15, np.nan, 17, np.nan],
    '特征4': [18, 19, 20, 21, 22, 23]
}
df = pd.DataFrame(data)
print("删除之前的df：")
print(df)

# 计算每列的缺失值数量
missing_values = df.isnull().sum()
print(f"每列的缺失值数量\n{missing_values}\n")

# 找到缺失值最多的列
column_with_most_missing_values = missing_values.idxmax()
print(f"缺失值最多的列:{column_with_most_missing_values}\n")

# 删除缺失值最多的列
df.drop(column_with_most_missing_values, axis=1, inplace=True)

# 将预处理后的数据集转换为张量格式
tensor = torch.tensor(df.to_numpy(dtype=float))

print("预处理后的数据集张量格式：")
print(tensor)


删除之前的df：
   特征1   特征2   特征3  特征4
0  1.0   7.0  13.0   18
1  2.0   8.0  14.0   19
2  NaN   9.0  15.0   20
3  4.0   NaN   NaN   21
4  5.0  11.0  17.0   22
5  6.0  12.0   NaN   23
每列的缺失值数量
特征1    1
特征2    1
特征3    2
特征4    0
dtype: int64

缺失值最多的列:特征3

预处理后的数据集张量格式：
tensor([[ 1.,  7., 18.],
        [ 2.,  8., 19.],
        [nan,  9., 20.],
        [ 4., nan, 21.],
        [ 5., 11., 22.],
        [ 6., 12., 23.]], dtype=torch.float64)
